In [14]:
from gensim.models.keyedvectors import KeyedVectors

# load word vectors
word_vectors = KeyedVectors.load_word2vec_format(
    'word2vec')

# create list of projection data
# tuple (label, vector)
labels = word_vectors.index_to_key
vectors = [word_vectors[label] for label in labels]

projection_data = [(label, vector) for label, vector in zip(labels, vectors)]

In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorboard.plugins import projector

def create_projection(projection_data, path='./logs/vocab'):
    meta_file = 'metadata.tsv'
    samples = len(projection_data)
    vector_dim = len(projection_data[0][1])
    projection_matrix = np.zeros((samples, vector_dim))

    # write meta file with labels, create projection_matrix
    with open(os.path.join(path, meta_file), 'w') as f:
        for i, row in enumerate(projection_data):
            label, vector = row[0], row[1]
            projection_matrix[i] = vector
            f.write(f"{label}\n")

    weights = tf.Variable(
        projection_matrix, trainable=False, name='word_embeddings'
    )

    checkpoint = tf.train.Checkpoint(embedding=weights)
    checkpoint.save(os.path.join(path, "embedding.ckpt"))
    
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
    embedding.metadata_path = meta_file
    projector.visualize_embeddings(path, config)

create_projection(projection_data)

In [16]:
%load_ext tensorboard
%tensorboard --logdir="./tensorboard/"

Reusing TensorBoard on port 6006 (pid 20440), started 1 day, 2:19:49 ago. (Use '!kill 20440' to kill it.)